# Download and build the knowledge base

In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [51]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame(documents, columns=['course','section', 'question' ,'text'])
df.head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


## Vectorized
## Vector space:
- Turn docs into vectors
- term-documents matrix
  - rows: documents
  - columns: words

### Bag of words

In [4]:

from sklearn.feature_extraction.text import CountVectorizer

In [5]:
cv = CountVectorizer(stop_words='english',min_df=5)

In [6]:
cv.fit(df.text)

CountVectorizer(min_df=5, stop_words='english')

In [7]:
name = cv.get_feature_names_out()

In [8]:
docs_example = [
    "Course starts on 15th Jan 2024",
    "Prerequisites listed on GitHub",
    "Submit homeworks after start date",
    "Registration not required for participation",
    "Setup Google Cloud and Python before course"
]
cvx = CountVectorizer(stop_words='english')
cvx.fit(docs_example)
x = cvx.transform(docs_example)

In [9]:
cvx.get_feature_names_out()

array(['15th', '2024', 'cloud', 'course', 'date', 'github', 'google',
       'homeworks', 'jan', 'listed', 'participation', 'prerequisites',
       'python', 'registration', 'required', 'setup', 'start', 'starts',
       'submit'], dtype=object)

In [10]:
dfx = pd.DataFrame(x.todense(),columns=cvx.get_feature_names_out())
dfx.T

,0,1,2,3,4
15th,1,0,0,0,0
2024,1,0,0,0,0
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
github,0,1,0,0,0
google,0,0,0,0,1
homeworks,0,0,1,0,0
jan,1,0,0,0,0
listed,0,1,0,0,0


In [11]:
X = cv.transform(df.text)
bw_docs = pd.DataFrame(X.toarray(),columns=name).T
bw_docs

,0,1,2,3,4,5,6,7,8,9,...,938,939,940,941,942,943,944,945,946,947
01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yes,0,0,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
yml,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
youtube,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
zip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### TF-IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
cv = TfidfVectorizer(stop_words='english',min_df=5)
cv.fit(df.text)
name = cv.get_feature_names_out()
X = cv.transform(df.text)
tf_docs = pd.DataFrame(X.toarray(), columns=name).T
tf_docs.round(2)


,0,1,2,3,4,5,6,7,8,9,...,938,939,940,941,942,943,944,945,946,947
01,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
02,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
03,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
04,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
05,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yes,0.0,0.00,0.28,0.0,0.0,0.00,0.21,0.2,0.15,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
yml,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.11,0.0,0.0,0.0,0.00
youtube,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.15,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
zip,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00


In [14]:
name

array(['01', '02', '03', ..., 'youtube', 'zip', 'zoomcamp'],
      shape=(1333,), dtype=object)

### Vectorized query

In [15]:
query = "Do I need to know Python to sign up for the January course?"
q = cv.transform([query])
q.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 1333))

In [16]:
query_dict = dict(zip(name, q.toarray()[0]))
query_dict

{'01': np.float64(0.0),
 '02': np.float64(0.0),
 '03': np.float64(0.0),
 '04': np.float64(0.0),
 '05': np.float64(0.0),
 '06': np.float64(0.0),
 '09': np.float64(0.0),
 '10': np.float64(0.0),
 '100': np.float64(0.0),
 '11': np.float64(0.0),
 '12': np.float64(0.0),
 '127': np.float64(0.0),
 '13': np.float64(0.0),
 '14': np.float64(0.0),
 '15': np.float64(0.0),
 '16': np.float64(0.0),
 '17': np.float64(0.0),
 '19': np.float64(0.0),
 '1st': np.float64(0.0),
 '20': np.float64(0.0),
 '2019': np.float64(0.0),
 '2020': np.float64(0.0),
 '2021': np.float64(0.0),
 '2022': np.float64(0.0),
 '2023': np.float64(0.0),
 '2024': np.float64(0.0),
 '21': np.float64(0.0),
 '22': np.float64(0.0),
 '24': np.float64(0.0),
 '25': np.float64(0.0),
 '2pacx': np.float64(0.0),
 '30': np.float64(0.0),
 '35': np.float64(0.0),
 '403': np.float64(0.0),
 '42': np.float64(0.0),
 '50': np.float64(0.0),
 '5000': np.float64(0.0),
 '5431': np.float64(0.0),
 '5432': np.float64(0.0),
 '60': np.float64(0.0),
 '600': np.floa

### Compute docs and query cosine similarity

In [17]:
X.dot(q.T).todense()

matrix([[0.19464486],
        [0.        ],
        [0.        ],
        [0.06011641],
        [0.04932915],
        [0.        ],
        [0.        ],
        [0.13477565],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.15899187],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.07431408],
        [0.        ],
        [0.        ],
        [0.05779673],
        [0.07243428],
        [0.        ],
        [0.05174293],
        [0.16373635],
        [0.08076031],
        [0.        ],
        [0.09755254],
        [0.        ],
        [0.21069625],
        [0.12067781],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.06381749],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.00910541],
        [0.02835681],
        [0.05480112],
        [0.        ],
        [0.        ],
        [0.        ],
        [0

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
score = cosine_similarity(X,q).flatten()
score

array([0.19464486, 0.        , 0.        , 0.06011641, 0.04932915,
       0.        , 0.        , 0.13477565, 0.        , 0.        ,
       0.        , 0.15899187, 0.        , 0.        , 0.        ,
       0.07431408, 0.        , 0.        , 0.05779673, 0.07243428,
       0.        , 0.05174293, 0.16373635, 0.08076031, 0.        ,
       0.09755254, 0.        , 0.21069625, 0.12067781, 0.        ,
       0.        , 0.        , 0.        , 0.06381749, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00910541,
       0.02835681, 0.05480112, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.02469964, 0.05129386, 0.06013439,
       0.05252658, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.04169018, 0.        , 0.        , 0.        , 0.0075293 ,
       0.        , 0.        , 0.01971463, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [19]:
# sort the cosine similarity score
import numpy as np
sort_score = np.argsort(score)
sort_score

array([718, 699, 700, 701, 703, 704, 707, 710, 711, 712, 717, 698, 476,
       477, 478, 479, 480, 481, 482, 483, 486, 487, 833, 820, 821, 822,
       823, 824, 825, 827, 828, 830, 832, 488, 834, 467, 468, 471, 472,
       473, 474, 475, 697, 526, 514, 515, 516, 517, 518, 519, 520, 523,
       524, 525, 513, 527, 528, 530, 532, 533, 534, 535, 536, 537, 538,
       499, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 819, 501,
       504, 505, 506, 507, 508, 509, 510, 512, 864, 386, 851, 853, 854,
       855, 859, 860, 861, 862, 863, 385, 865, 387, 389, 390, 392, 397,
       399, 400, 402, 404, 369, 358, 359, 360, 361, 362, 363,  32, 366,
       367, 368, 405, 370, 371, 376, 377, 379, 380, 382, 383, 384, 442,
       839, 840, 841, 843, 846, 847, 848, 850, 437, 441, 836, 443, 444,
       447, 453, 460, 461, 462, 463, 466, 420, 407, 408, 409, 410, 412,
       414, 416, 417, 418, 419, 542, 421, 422, 423, 426, 427, 428, 429,
       430, 432, 654, 643, 644, 645, 646, 647, 649, 650, 651, 65

In [20]:
# sort the score value and return its index(ascending), the biggest value's index would be at the last
sort_score[-5:]

array([764,  27, 806, 577, 445])

In [21]:
arr = np.array([111,89, 3, 1, 4, 1, 5, 99])
sorted_indices = np.argsort(arr)
print(sorted_indices) 

[3 5 2 4 6 1 7 0]


In [22]:
# tf-idf capture about "python learning"
df.iloc[445].text

'Check this article. If you know everything in this article, you know enough. If you don’t, read the article and join the coursIntroduction to Pythone too :)\nIntroduction to Python – Machine Learning Bookcamp\nYou can follow this English course from the OpenClassrooms e-learning platform, which is free and covers the python basics for data analysis: Learn Python Basics for Data Analysis - OpenClassrooms . It is important to know some basics such as: how to run a Jupyter notebook, how to import libraries (and what libraries are), how to declare a variable (and what variables are) and some important operations regarding data analysis.\n(Mélanie Fouesnard)'

### Vectorized all the fields of docs

In [23]:
fields = ['section', 'question', 'text']

In [24]:
matrices = {}
vectorizers = {}
for f in fields:
    cv = TfidfVectorizer(stop_words='english',min_df=5)
    X = cv.fit_transform(df[f])
    matrices[f] = X
    vectorizers[f] = cv
    

In [25]:
matrices

{'section': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 3090 stored elements and shape (948, 66)>,
 'question': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 3431 stored elements and shape (948, 291)>,
 'text': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 23808 stored elements and shape (948, 1333)>}

In [26]:
vectorizers

{'section': TfidfVectorizer(min_df=5, stop_words='english'),
 'question': TfidfVectorizer(min_df=5, stop_words='english'),
 'text': TfidfVectorizer(min_df=5, stop_words='english')}

### compute cosine similarity between each field and the query and sum them up.

In [27]:
# create a size as big as docs zero matrix for accumelate fields' score
n = len(df)
score = np.zeros(n)

query = "I justed discovered the course, is it too late to join?"

# for each field, use its vectorizer and matrix to calculate cosine similarity with query
for f in fields:
    q = vectorizers[f].transform([query])
    X = matrices[f]
    f_score = cosine_similarity(X,q).flatten()
    score += f_score

In [28]:
top_5 = np.argsort(score)[-5:]
df.iloc[top_5]

,course,section,question,text
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
448,machine-learning-zoomcamp,General course-related questions,I’m new to Slack and can’t find the course cha...,Here’s how you join a in Slack: https://slack....
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...


In [29]:
# limit the result in a specfic field
filter = {'course':'data-engineering-zoomcamp'}



for field, value in filter.items():
    mask = (df[field] == value).astype(int).values
    score *= mask
top_5 = np.argsort(score)[-5:]
df.iloc[top_5]

,course,section,question,text
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...
34,data-engineering-zoomcamp,General course-related questions,How can we contribute to the course?,Star the repo! Share it with friends if you fi...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...


In [30]:
# consider each field's weight
boosts = {'question':3, 'text':0.5}
score = np.zeros(len(df))
for f in fields:
    q = vectorizers[f].transform([query])
    X = matrices[f]
    f_score = cosine_similarity(X,q).flatten()
    boost = boosts.get(f,1.0)
    score = score + boost * f_score 
for field, value in filter.items():
    mask = (df[field]==value).astype(int).values
    score = mask * score
# top_5 = np.argsort(score)[-5:]
# df.iloc[top_5]
score

array([3.31803708, 3.49512426, 2.70735166, 2.75775375, 3.49512426,
       3.49512426, 1.93689291, 3.59330081, 2.67242848, 3.49512426,
       3.10198469, 2.39100617, 0.49512426, 0.49512426, 0.49512426,
       0.54925793, 0.49512426, 2.63772182, 0.53722596, 0.49512426,
       0.49512426, 0.49512426, 0.61439707, 0.55395365, 0.49512426,
       0.49512426, 0.49512426, 0.64860474, 0.53471899, 0.49512426,
       0.49512426, 0.49512426, 0.49512426, 1.76696195, 3.49512426,
       1.72080809, 0.49512426, 0.49512426, 0.49512426, 0.49512426,
       0.49512426, 1.96968136, 0.49512426, 0.49512426, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00548468,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [31]:
idx = np.argsort(-score)[:5]

In [32]:
df.iloc[idx]

,course,section,question,text
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
5,data-engineering-zoomcamp,General course-related questions,Course - how many Zoomcamps in a year?,"There are 3 Zoom Camps in a year, as of 2024. ..."
34,data-engineering-zoomcamp,General course-related questions,How can we contribute to the course?,Star the repo! Share it with friends if you fi...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


In [33]:
s = np.array([0.2, 0.8, 0.5, 0.1, 0.9, 0.3])

id1 = np.argsort(s)[-5:] #先對s進行升密排列在取最後五個
print(s[id1])

id2 = np.argsort(-s)[:5] #先對s進行降序排列在取前五個
print(s[id2])

[0.2 0.3 0.5 0.8 0.9]
[0.9 0.8 0.5 0.3 0.2]


## Putting it all together

In [40]:
class TextSearch:
    def __init__(self,text_fields):
        self.text_fields = text_fields
        self.matrices = {}
        self.vectorizers = {}

    def fit(self, records, vectorizer_params={}):
        self.df = pd.DataFrame(records)

        for f in self.text_fields:
            cv = TfidfVectorizer(**vectorizer_params)
            X = cv.fit_transform(self.df[f])
            self.matrices[f] = X
            self.vectorizers[f] = cv

    def search(self, query, n_results=10, boost={}, filters={}):
        score = np.zeros(len(self.df))

        for f in self.text_fields:
            b = boosts.get(f,1.0)
            q = self.vectorizers[f].transform([query])
            s = cosine_similarity(self.matrices[f], q).flatten()
            score = score + b * s

        for field, value in filter.items():
            mask = (self.df[field] == value).values
            score = score * mask

        idx = np.argsort(-score)[:n_results]
        results = self.df.iloc[idx]
        return results.to_dict(orient='records')


In [42]:
index = TextSearch(
    ['section', 'question', 'text']
)
index.fit(documents)

index.search(
    query = 'I just signed up. It is too late to join the course?',
    n_results = 5,
    boost = {'question' : 3.0},
    filters = {'course': 'data-engineering-zoomcamp'}
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## SVD singular value decomposition 奇異值分解
 Latent Semantic Analysis 一種降維技術，用來把原始的「詞 × 文件」矩陣轉換成一個更小的「語意空間」，讓語意相近但字面不同的詞或文件可以靠近彼此。

In [45]:
from sklearn.decomposition import TruncatedSVD

X = matrices['text']
cv = vectorizers['text']

# 原本為948 * 1333

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 23808 stored elements and shape (948, 1333)>

In [46]:
svd = TruncatedSVD(n_components=16)
X_emb = svd.fit_transform(X)

In [47]:
X_emb[0] # 16 dimension

array([ 0.09652719, -0.08227852, -0.10235345, -0.07815947,  0.06818714,
       -0.06184413,  0.01961047, -0.14155493, -0.23950671,  0.28425807,
        0.07850089,  0.0353366 , -0.14830127,  0.05578769,  0.02250833,
       -0.03534514])

In [48]:
query = 'I just signed up. It is too late to join the course?'
Q = cv.transform([query])
Q_emb = svd.transform(Q)
Q_emb[0]

array([ 0.05790177, -0.03861948, -0.05713458, -0.02784242,  0.04022505,
       -0.06268733,  0.00914148, -0.08890379, -0.17002683,  0.17753312,
        0.05962452,  0.04597167, -0.09999198,  0.03465096,  0.03715293,
       -0.00860512])

In [50]:
score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:5]
df.iloc[idx]

,course,section,question,text
764,machine-learning-zoomcamp,Projects (Midterm and Capstone),What If I submitted only two projects and fail...,If you have submitted two projects (and peer-r...
436,machine-learning-zoomcamp,General course-related questions,Is it going to be live? When?,"The course videos are pre-recorded, you can st..."
449,machine-learning-zoomcamp,General course-related questions,The course has already started. Can I still jo...,"Yes, you can. You won’t be able to submit some..."
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
814,mlops-zoomcamp,+-General course questions,What if my answer is not exactly the same as t...,Please choose the closest one to your answer. ...


## Workflow

1. Prepare dataset, sort out as a dictionary eg,.{'section': [section], 'question':[question], 'text':[content]}
2. content vectorization (bag, tfidf) to create knowledge database
3. vectorized query
4. compute the score of cosine similarity between content and query and flatten (turn it into a 1D matrtix) Sementic search
5. Rank the order of score(np.argsort(score))
6. vecotrize all the fields of dataset(including all columns, compute cosine similarity between each field and the query and sum them up.(score = np.zeros(n))
7. consider different fields have different weight impact to score
   boost = {'question':3, 'text':0.5}
   for f in fields:
    q = vectorizers[f].transform([query])
    X = matrices[f]
    f_score = cosine_similarity(X,q).flatten()
    boost = boosts.get(f,1.0)
    score = score + boost * f_score
   
9. Consider filter to limit the result in specific field (filter={'section':'dze'})
    for field, value in filter.items():
    mask = (df[field] == value).astype(int).values
    score *= mask
10. Consider decompostion sparse matrix(LSA降維改進效率SVD, NMF)

## BERT
Not only capture sementic similarity but also work orders

In [53]:
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [54]:
texts = [
    "Yes, we will keep all the materials after the course finishes.",
    "You can follow the course at your own pace after it finishes."
]
encode_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
encode_input

{'input_ids': tensor([[  101,  2748,  1010,  2057,  2097,  2562,  2035,  1996,  4475,  2044,
          1996,  2607, 12321,  1012,   102],
        [  101,  2017,  2064,  3582,  1996,  2607,  2012,  2115,  2219,  6393,
          2044,  2009, 12321,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [55]:
with torch.no_grad():
    outputs = model(**encode_input)
    hidden_states = outputs.last_hidden_state

In [57]:
hidden_states.shape

torch.Size([2, 15, 768])

In [58]:
sentence_embeddings = hidden_states.mean(dim=1)
sentence_embeddings

tensor([[ 0.3600, -0.1607,  0.3545,  ...,  0.0429,  0.0348, -0.0382],
        [ 0.3217, -0.4399,  0.1317,  ..., -0.1010, -0.3796,  0.4087]])